<a href="https://colab.research.google.com/github/seanreed1111/colab-demos/blob/master/test_json_struct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qqq jmespath deepdiff python-jsonschema-objects

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 220.4 kB/s eta 0:00:00


In [2]:
import json
import jmespath
# from collections import namedtuple
# from copy import deepcopy
# from json import JSONEncoder
from deepdiff import DeepDiff
# import python_jsonschema_objects as pjs


In [16]:
deepdiff_options = dict(ignore_order=True, report_repetition=True, ignore_string_case=True)

In [3]:
two_double_jacks_no_pickles_no_onions = {'categories': [{'items': [{'item_aliases': [], 'item_name': 'Double Jack', 'modifications': ["no pickles", "no onions"], 'quantity': 2, 'special_requests': []}], 'name': 'Burger'}], 'combos': []};
two_double_jacks_no_onions_no_pickles = {'categories': [{'items': [{'item_aliases': [], 'item_name': 'Double Jack', 'modifications': ["no onions", "no pickles"], 'quantity': 2, 'special_requests': []}], 'name': 'Burger'}], 'combos': []};


In [10]:
actual = two_double_jacks_no_pickles_no_onions
expected = two_double_jacks_no_onions_no_pickles

In [11]:
actual == expected # False since modifications are not in same order

False

In [12]:
# DeepDiff returns an empty dict-like object if the two items are 'identical'
DeepDiff(two_double_jacks_no_pickles_no_onions, two_double_jacks_no_onions_no_pickles, **deepdiff_options)

{}

In [64]:
# Important! the options are not optional for our use case
DeepDiff(two_double_jacks_no_pickles_no_onions, two_double_jacks_no_onions_no_pickles)

{'values_changed': {"root['categories'][0]['items'][0]['modifications'][0]": {'new_value': 'no onions',
   'old_value': 'no pickles'},
  "root['categories'][0]['items'][0]['modifications'][1]": {'new_value': 'no pickles',
   'old_value': 'no onions'}}}

### New Example

In [4]:
two_double_jacks_no_pickles = {'categories': [{'items': [{'item_aliases': [], 'item_name': 'Double Jack', 'modifications': ["no pickles"], 'quantity': 2, 'special_requests': []}], 'name': 'Burger'}], 'combos': []};
two_double_jacks_no_onions = {'categories': [{'items': [{'item_aliases': [], 'item_name': 'Double Jack', 'modifications': ["no onions"], 'quantity': 2, 'special_requests': []}], 'name': 'Burger'}], 'combos': []};


In [13]:
#deepdiff reports what must be added or removed from the 2nd arg to make it identical to the first
DeepDiff(two_double_jacks_no_pickles, two_double_jacks_no_onions, **deepdiff_options)

{'iterable_item_added': {"root['categories'][0]['items'][0]['modifications'][0]": 'no onions'},
 'iterable_item_removed': {"root['categories'][0]['items'][0]['modifications'][0]": 'no pickles'}}

In [14]:
DeepDiff(two_double_jacks_no_pickles, two_double_jacks_no_onions, **deepdiff_options).to_json()

'{"iterable_item_added": {"root[\'categories\'][0][\'items\'][0][\'modifications\'][0]": "no onions"}, "iterable_item_removed": {"root[\'categories\'][0][\'items\'][0][\'modifications\'][0]": "no pickles"}}'

In [56]:
from copy import deepcopy
from deepdiff import DeepDiff

class NewStruct():
    deepdiff_options = dict(ignore_order=True, report_repetition=True)
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __repr__(self):
        return str(self.__dict__)

    def as_dict(self):
      return self.__dict__

    def as_json(self):
      return json.dumps(self.__dict__)

    def diff(self, other):
        if isinstance(other, NewStruct):
          return DeepDiff(self.as_dict(), other.as_dict(), **NewStruct.deepdiff_options)
        elif isinstance(other, dict):
            return DeepDiff(self.as_dict(), other, **NewStruct.deepdiff_options)
        else:
          return DeepDiff(self, other, **NewStruct.deepdiff_options )

    def get_categories(self):
      for category in deepcopy(self.categories):
        yield category

    def get_items(self):
      for category in deepcopy(self.categories):
        for item in category['items']:
          yield category.name, item

    def get_combos(self):
      for combo in deepcopy(self.combos):
        yield combo

    def get_category_names(self):
      for category in deepcopy(self.categories):
        yield category.name


    def get_item_names(self):
      for category in deepcopy(self.categories):
        for item in category['items']:
          yield item.item_name

    def __eq__(self, other):
      # performs equality checks against NewStruct objs, str, and dict
        if isinstance(other, NewStruct):
            # diff returns an empty dict if objects are equal
            return not self.diff(other)
        elif isinstance(other, dict):
            return not DeepDiff(self.as_dict(), other, **NewStruct.deepdiff_options)
        else:
          return False

In [57]:
a = NewStruct(**two_double_jacks_no_pickles_no_onions)
b = NewStruct(**two_double_jacks_no_onions_no_pickles)
print(a, "\n", b)

{'categories': [{'items': [{'item_aliases': [], 'item_name': 'Double Jack', 'modifications': ['no pickles', 'no onions'], 'quantity': 2, 'special_requests': []}], 'name': 'Burger'}], 'combos': []} 
 {'categories': [{'items': [{'item_aliases': [], 'item_name': 'Double Jack', 'modifications': ['no onions', 'no pickles'], 'quantity': 2, 'special_requests': []}], 'name': 'Burger'}], 'combos': []}


In [58]:
a.categories

[{'items': [{'item_aliases': [],
    'item_name': 'Double Jack',
    'modifications': ['no pickles', 'no onions'],
    'quantity': 2,
    'special_requests': []}],
  'name': 'Burger'}]

In [60]:
a.get_categories(), a.get_category_names()

(<generator object NewStruct.get_categories at 0x7b8ef9995690>,
 <generator object NewStruct.get_category_names at 0x7b8ef9995af0>)

In [42]:
a.get_items(), b.get_items()

(<generator object NewStruct.get_items at 0x7b8ef9994c10>,
 <generator object NewStruct.get_items at 0x7b8ef99945f0>)

In [43]:
a == b

True

In [44]:
a.as_json()

'{"categories": [{"items": [{"item_aliases": [], "item_name": "Double Jack", "modifications": ["no pickles", "no onions"], "quantity": 2, "special_requests": []}], "name": "Burger"}], "combos": []}'

In [45]:
b.as_json()

'{"categories": [{"items": [{"item_aliases": [], "item_name": "Double Jack", "modifications": ["no onions", "no pickles"], "quantity": 2, "special_requests": []}], "name": "Burger"}], "combos": []}'

In [46]:
a.as_dict()

{'categories': [{'items': [{'item_aliases': [],
     'item_name': 'Double Jack',
     'modifications': ['no pickles', 'no onions'],
     'quantity': 2,
     'special_requests': []}],
   'name': 'Burger'}],
 'combos': []}

In [47]:
b.as_dict()

{'categories': [{'items': [{'item_aliases': [],
     'item_name': 'Double Jack',
     'modifications': ['no onions', 'no pickles'],
     'quantity': 2,
     'special_requests': []}],
   'name': 'Burger'}],
 'combos': []}

In [48]:
a.diff(b)

{}

In [49]:
a.diff(b.as_dict())

{}

In [50]:
a.diff(b.as_json())

{'type_changes': {'root': {'old_type': method,
   'new_type': str,
   'old_value': <bound method NewStruct.as_json of {'categories': [{'items': [{'item_aliases': [], 'item_name': 'Double Jack', 'modifications': ['no pickles', 'no onions'], 'quantity': 2, 'special_requests': []}], 'name': 'Burger'}], 'combos': []}>,
   'new_value': '{"categories": [{"items": [{"item_aliases": [], "item_name": "Double Jack", "modifications": ["no onions", "no pickles"], "quantity": 2, "special_requests": []}], "name": "Burger"}], "combos": []}'}}}

In [79]:
bigorder_salad_combos =  {'categories': [{'name': 'Salad', 'items': [{'item_aliases': [], 'item_name': 'Garden Salad with Grilled Chicken', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Southwest Salad with Crispy Chicken', 'modifications': [], 'quantity': 1, 'special_requests': []}]}], 'combos': [{'combo_aliases': [], 'items': [{'item_aliases': [], 'item_name': 'Jacks Spicy Chicken with Cheese', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'French Fries Small', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Small Coca-Cola', 'modifications': [], 'quantity': 1, 'special_requests': []}], 'name': 'Jacks Spicy Chicken with Cheese Combo'}, {'combo_aliases': ['#11'], 'items': [{'item_aliases': [], 'item_name': 'Homestyle Ranch Chicken Club', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'French Fries Small', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Small Coca-Cola', 'modifications': [], 'quantity': 1, 'special_requests': []}], 'name': 'Homestyle Ranch Chicken Club Combo'}, {'combo_aliases': ['#13'], 'items': [{'item_aliases': [], 'item_name': 'Grilled Chicken Sandwich', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'French Fries Small', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Small Coca-Cola', 'modifications': [], 'quantity': 1, 'special_requests': []}], 'name': 'Grilled Chicken Sandwich Combo'}, {'combo_aliases': ['#14'], 'items': [{'item_aliases': [], 'item_name': 'Crispy Chicken Strips 3 pc', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'French Fries Small', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Small Coca-Cola', 'modifications': [], 'quantity': 1, 'special_requests': []}], 'name': 'Crispy Chicken Strips Combo'}, {'combo_aliases': ['#15'], 'items': [{'item_aliases': [], 'item_name': 'Chicken Nuggets 4 pc', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'French Fries Small', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Small Coca-Cola', 'modifications': [], 'quantity': 1, 'special_requests': []}], 'name': 'Chicken Nuggets 8 pc Combo'}, {'combo_aliases': ['#16'], 'items': [{'item_aliases': ['Teriyaki Bowl'], 'item_name': 'Chicken Teriyaki Bowl', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'French Fries Small', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Small Coca-Cola', 'modifications': [], 'quantity': 1, 'special_requests': []}], 'name': 'Chicken Teriyaki Bowl Combo'}, {'combo_aliases': ['#17'], 'items': [{'item_aliases': ['Chicken Fijita'], 'item_name': 'Chicken Fajita Pita', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'French Fries Small', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Small Coca-Cola', 'modifications': [], 'quantity': 1, 'special_requests': []}], 'name': 'Chicken Fajita Pita Combo'}]}

In [78]:
bigorder_chicken_burgers_combo = {'categories': [{'name': 'Burger', 'items': [{'item_aliases': [], 'item_name': 'Classic Smashed Jack', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Ultimate Cheeseburger', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Classic Buttery Jack', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Jumbo Jack Cheeseburger', 'modifications': [], 'quantity': 1, 'special_requests': []}]}, {'name': 'Chicken', 'items': [{'item_aliases': ['Cluck'], 'item_name': 'Cluck Chicken Sandwich', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Homestyle Ranch Chicken Club', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Jacks Spicy Chicken with Cheese', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Grilled Chicken Sandwich', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': ['Teriyaki Bowl'], 'item_name': 'Chicken Teriyaki Bowl', 'modifications': [], 'quantity': 1, 'special_requests': []}]}, {'name': 'Salad', 'items': [{'item_aliases': [], 'item_name': 'Garden Salad with Grilled Chicken', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Southwest Salad with Grilled Chicken', 'modifications': [], 'quantity': 1, 'special_requests': []}]}, {'name': 'Side', 'items': [{'item_aliases': [], 'item_name': 'Classic Grilled Jack Wrap', 'modifications': [], 'quantity': 1, 'special_requests': []}]}], 'combos': [{'combo_aliases': ['#1'], 'items': [{'item_aliases': [], 'item_name': 'Sourdough Jack', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'French Fries Small', 'modifications': [], 'quantity': 1, 'special_requests': []}, {'item_aliases': [], 'item_name': 'Large Sprite', 'modifications': [], 'quantity': 1, 'special_requests': []}], 'name': 'Sourdough Jack Combo'}]}

In [80]:
bo_1 = bigorder_salad_combos
bo_2 = bigorder_chicken_burgers_combo

### bigorder_copy

In [66]:
bigorder_copy = {
   "categories":[
      {
         "name":"Salad",
         "items":[
            {
               "item_aliases":[

               ],
               "item_name":"Garden Salad with Grilled Chicken",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"Southwest Salad with Crispy Chicken",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            }
         ]
      }
   ],
   "combos":[
      {
         "combo_aliases":[

         ],
         "items":[
            {
               "item_aliases":[

               ],
               "item_name":"Jacks Spicy Chicken with Cheese",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"French Fries Small",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"Small Coca-Cola",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            }
         ],
         "name":"Jacks Spicy Chicken with Cheese Combo"
      },
      {
         "combo_aliases":[
            "#11"
         ],
         "items":[
            {
               "item_aliases":[

               ],
               "item_name":"Homestyle Ranch Chicken Club",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"French Fries Small",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"Small Coca-Cola",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            }
         ],
         "name":"Homestyle Ranch Chicken Club Combo"
      },
      {
         "combo_aliases":[
            "#13"
         ],
         "items":[
            {
               "item_aliases":[

               ],
               "item_name":"Grilled Chicken Sandwich",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"French Fries Small",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"Small Coca-Cola",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            }
         ],
         "name":"Grilled Chicken Sandwich Combo"
      },
      {
         "combo_aliases":[
            "#14"
         ],
         "items":[
            {
               "item_aliases":[

               ],
               "item_name":"Crispy Chicken Strips 3 pc",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"French Fries Small",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"Small Coca-Cola",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            }
         ],
         "name":"Crispy Chicken Strips Combo"
      },
      {
         "combo_aliases":[
            "#15"
         ],
         "items":[
            {
               "item_aliases":[

               ],
               "item_name":"Chicken Nuggets 4 pc",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"French Fries Small",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"Small Coca-Cola",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            }
         ],
         "name":"Chicken Nuggets 8 pc Combo"
      },
      {
         "combo_aliases":[
            "#16"
         ],
         "items":[
            {
               "item_aliases":[
                  "Teriyaki Bowl"
               ],
               "item_name":"Chicken Teriyaki Bowl",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"French Fries Small",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"Small Coca-Cola",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            }
         ],
         "name":"Chicken Teriyaki Bowl Combo"
      },
      {
         "combo_aliases":[
            "#17"
         ],
         "items":[
            {
               "item_aliases":[
                  "Chicken Fijita"
               ],
               "item_name":"Chicken Fajita Pita",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"French Fries Small",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            },
            {
               "item_aliases":[

               ],
               "item_name":"Small Coca-Cola",
               "modifications":[

               ],
               "quantity":1,
               "special_requests":[

               ]
            }
         ],
         "name":"Chicken Fajita Pita Combo"
      }
   ]
}

### begin testing with bigorder_copy and jmespath

In [75]:
jmespath.search("categories[].name",bigorder_copy)

['Salad']

In [76]:
jmespath.search("combos[].name",bigorder_copy)

['Jacks Spicy Chicken with Cheese Combo',
 'Homestyle Ranch Chicken Club Combo',
 'Grilled Chicken Sandwich Combo',
 'Crispy Chicken Strips Combo',
 'Chicken Nuggets 8 pc Combo',
 'Chicken Teriyaki Bowl Combo',
 'Chicken Fajita Pita Combo']

In [77]:
jmespath.search("combos[]",bigorder_copy)

[{'combo_aliases': [],
  'items': [{'item_aliases': [],
    'item_name': 'Jacks Spicy Chicken with Cheese',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'French Fries Small',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'Small Coca-Cola',
    'modifications': [],
    'quantity': 1,
    'special_requests': []}],
  'name': 'Jacks Spicy Chicken with Cheese Combo'},
 {'combo_aliases': ['#11'],
  'items': [{'item_aliases': [],
    'item_name': 'Homestyle Ranch Chicken Club',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'French Fries Small',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'Small Coca-Cola',
    'modifications': [],
    'quantity': 1,
    'special_requests': []}],
  'name': 'Homestyle Ranch Chicken Club Comb

In [82]:
jmespath.search("categories[].name",bo_1)

['Salad']

In [88]:
jmespath.search("categories[].name",bo_2)

['Burger', 'Chicken', 'Salad', 'Side']

In [89]:
jmespath.search("categories[]",bo_1)

[{'name': 'Salad',
  'items': [{'item_aliases': [],
    'item_name': 'Garden Salad with Grilled Chicken',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'Southwest Salad with Crispy Chicken',
    'modifications': [],
    'quantity': 1,
    'special_requests': []}]}]

In [90]:
jmespath.search("categories[]",bo_2)

[{'name': 'Burger',
  'items': [{'item_aliases': [],
    'item_name': 'Classic Smashed Jack',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'Ultimate Cheeseburger',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'Classic Buttery Jack',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'Jumbo Jack Cheeseburger',
    'modifications': [],
    'quantity': 1,
    'special_requests': []}]},
 {'name': 'Chicken',
  'items': [{'item_aliases': ['Cluck'],
    'item_name': 'Cluck Chicken Sandwich',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'Homestyle Ranch Chicken Club',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'Jacks Spicy Chicken with Cheese',
    'modificat

In [84]:
jmespath.search("combos[]",bo_1)

[{'combo_aliases': [],
  'items': [{'item_aliases': [],
    'item_name': 'Jacks Spicy Chicken with Cheese',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'French Fries Small',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'Small Coca-Cola',
    'modifications': [],
    'quantity': 1,
    'special_requests': []}],
  'name': 'Jacks Spicy Chicken with Cheese Combo'},
 {'combo_aliases': ['#11'],
  'items': [{'item_aliases': [],
    'item_name': 'Homestyle Ranch Chicken Club',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'French Fries Small',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'Small Coca-Cola',
    'modifications': [],
    'quantity': 1,
    'special_requests': []}],
  'name': 'Homestyle Ranch Chicken Club Comb

In [87]:
jmespath.search("combos[]",bo_2)

[{'combo_aliases': ['#1'],
  'items': [{'item_aliases': [],
    'item_name': 'Sourdough Jack',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'French Fries Small',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'Large Sprite',
    'modifications': [],
    'quantity': 1,
    'special_requests': []}],
  'name': 'Sourdough Jack Combo'}]

### Begin testing with bigorder_copy and newstruct

In [67]:
big = NewStruct(**bigorder_copy) #fail

In [68]:
big.as_json()

'{"categories": [{"name": "Salad", "items": [{"item_aliases": [], "item_name": "Garden Salad with Grilled Chicken", "modifications": [], "quantity": 1, "special_requests": []}, {"item_aliases": [], "item_name": "Southwest Salad with Crispy Chicken", "modifications": [], "quantity": 1, "special_requests": []}]}], "combos": [{"combo_aliases": [], "items": [{"item_aliases": [], "item_name": "Jacks Spicy Chicken with Cheese", "modifications": [], "quantity": 1, "special_requests": []}, {"item_aliases": [], "item_name": "French Fries Small", "modifications": [], "quantity": 1, "special_requests": []}, {"item_aliases": [], "item_name": "Small Coca-Cola", "modifications": [], "quantity": 1, "special_requests": []}], "name": "Jacks Spicy Chicken with Cheese Combo"}, {"combo_aliases": ["#11"], "items": [{"item_aliases": [], "item_name": "Homestyle Ranch Chicken Club", "modifications": [], "quantity": 1, "special_requests": []}, {"item_aliases": [], "item_name": "French Fries Small", "modificati

In [69]:
big.categories

[{'name': 'Salad',
  'items': [{'item_aliases': [],
    'item_name': 'Garden Salad with Grilled Chicken',
    'modifications': [],
    'quantity': 1,
    'special_requests': []},
   {'item_aliases': [],
    'item_name': 'Southwest Salad with Crispy Chicken',
    'modifications': [],
    'quantity': 1,
    'special_requests': []}]}]

In [74]:
for category in big.get_categories():
  print(category)

AttributeError: 'dict' object has no attribute 'name'

In [ ]:
class Item:
  def __init__(self, **kwargs):
    self.item_aliases = kwargs['item_aliases']
    self.item_name = kwargs['item_name']
    # self.modifications = kwargs['modifications']
    self.modifications = set(kwargs['modifications'])
    self.quantity = kwargs['quantity']
    # self.special_requests = kwargs['special_requests']
    self.special_requests_set = set(kwargs['special_requests'])
    self.__dict__.update(kwargs)

  def __repr__(self):

    return str(self.__dict__)


class Category():
  def __init__(self, **kwargs):
    self.items = [item for item in kwargs['items']]
    self.items_set = set(self.items)
    self.__dict__.update(kwargs)

  def __repr__(self):
    return str(self.__dict__)


class NewStruct():
  def __init__(self, **kwargs):
    self.categories = [category for category in kwargs['categories']]
    self.combos = [combo for combo in kwargs['combos']]
    self.__dict__.update(kwargs)

  # def add_item(self, category, item):

  def __repr__(self):
    return str(self.__dict__)

In [ ]:
c = NewStruct(**two_double_jacks_no_pickles_no_onions);c

{'categories': [{'items': [{'item_aliases': [], 'item_name': 'Double Jack', 'modifications': ['no pickles', 'no onions'], 'quantity': 2, 'special_requests': []}], 'name': 'Burger'}], 'combos': []}

In [ ]:
c.categories

[{'items': [{'item_aliases': [],
    'item_name': 'Double Jack',
    'modifications': ['no pickles', 'no onions'],
    'quantity': 2,
    'special_requests': []}],
  'name': 'Burger'}]

In [ ]:
c.combos

[]

## start with item

In [ ]:
two_double_jacks_no_pickles_no_onions = {'categories': [{'items': [{'item_aliases': [], 'item_name': 'Double Jack', 'modifications': ["no pickles", "no onions"], 'quantity': 2, 'special_requests': []}], 'name': 'Burger'}], 'combos': []}; two_double_jacks_no_pickles_no_onions


https://github.com/jmespath/jmespath.py

*   https://www.zyte.com/blog/json-parsing-with-python/
*   https://pynative.com/python-convert-json-data-into-custom-python-object/



## Example 1 from https://pynative.com/python-convert-json-data-into-custom-python-object/

In [ ]:
import json
from collections import namedtuple
from json import JSONEncoder

def customStudentDecoder(studentDict):
    return namedtuple('X', studentDict.keys())(*studentDict.values())

#Assume you received this JSON response
studentJsonData = '{"rollNumber": 1, "name": "Emma"}'

# Parse JSON into an object with attributes corresponding to dict keys.
student = json.loads(studentJsonData, object_hook=customStudentDecoder)

print("After Converting JSON Data into Custom Python Object")
print(student.rollNumber, student.name)

After Converting JSON Data into Custom Python Object
1 Emma


In [ ]:
student

X(rollNumber=1, name='Emma')

## Example 2

In [ ]:
import json
from collections import namedtuple
from json import JSONEncoder

class Student:
    def __init__(self, rollNumber, name, marks):
        self.rollNumber, self.name, self.marks = rollNumber, name, marks

class Marks:
    def __init__(self, english, geometry):
        self.english, self.geometry = english, geometry

class StudentEncoder(JSONEncoder):
        def default(self, o):
            return o.__dict__

def customStudentDecoder(studentDict):
    return namedtuple('student', studentDict.keys())(*studentDict.values())

marks = Marks(82, 74)
student = Student(1, "Emma", marks)

# dumps() produces JSON in native str format. if you want to writ it in file use dump()
studentJson = json.dumps(student, indent=4, cls=StudentEncoder)
print("Student JSON")
print(studentJson)

# Parse JSON into an object with attributes corresponding to dict keys.
studObj = json.loads(studentJson, object_hook=customStudentDecoder)

print("After Converting JSON Data into Custom Python Object")
print(studObj.rollNumber, studObj.name, "\n")
print(studObj, "\n", studObj.marks.english, studObj.marks.geometry)

Student JSON
{
    "rollNumber": 1,
    "name": "Emma",
    "marks": {
        "english": 82,
        "geometry": 74
    }
}
After Converting JSON Data into Custom Python Object
1 Emma 

student(rollNumber=1, name='Emma', marks=student(english=82, geometry=74)) 
 82 74


## Example 3

In [ ]:
import json
from json import JSONEncoder
from types import SimpleNamespace as Namespace

class Student:
    def __init__(self, rollNumber, name, marks):
        self.rollNumber, self.name, self.marks = rollNumber, name, marks

class Marks:
    def __init__(self, english, geometry):
        self.english, self.geometry = english, geometry

class StudentEncoder(JSONEncoder):
        def default(self, o): return o.__dict__

marks = Marks(82, 74)
student = Student(1, "Emma", marks)

# dumps() produces JSON in native str format. if you want to write it in file use dump()
studentJsonData = json.dumps(student, indent=4, cls=StudentEncoder)
print("Student JSON")
print(studentJsonData)

# Parse JSON into an custom Student object.
studObj = json.loads(studentJsonData, object_hook=lambda d: Namespace(**d))
print("After Converting JSON Data into Custom Python Object using SimpleNamespace")
print(studObj.rollNumber, studObj.name, studObj.marks.english, studObj.marks.geometry)

Student JSON
{
    "rollNumber": 1,
    "name": "Emma",
    "marks": {
        "english": 82,
        "geometry": 74
    }
}
After Converting JSON Data into Custom Python Object using SimpleNamespace
1 Emma 82 74


## Example 4

In [ ]:
import json

class Student(object):
    def __init__(self, rollNumber, name, marks):
        self.rollNumber = rollNumber
        self.name = name
        self.marks = marks

def studentDecoder(obj):
    if '__type__' in obj and obj['__type__'] == 'Student':
        return Student(obj['rollNumber'], obj['name'], obj['marks'])
    return obj

studentObj = json.loads('{"__type__": "Student", "rollNumber":1, "name": "Ault kelly", "marks": 78}',
           object_hook=studentDecoder)

print("Type of decoded object from JSON Data")
print(type(studentObj))
print("Student Details")
print(studentObj.rollNumber, studentObj.name, studentObj.marks)

Type of decoded object from JSON Data
<class '__main__.Student'>
Student Details
1 Ault kelly 78


## Example 5

In [ ]:
import json
import jsonpickle
from json import JSONEncoder

class Student(object):
    def __init__(self, rollNumber, name, marks):
        self.rollNumber = rollNumber
        self.name = name
        self.marks = marks

class Marks(object):
    def __init__(self, english, geometry):
        self.english = english
        self.geometry = geometry

marks = Marks(82, 74)
student = Student(1, "Emma", marks)

print("Encode Object into JSON formatted Data using jsonpickle")
studentJSON = jsonpickle.encode(student)
print(studentJSON)

print("Decode and Convert JSON into Object using jsonpickle")
studentObject = jsonpickle.decode(studentJSON)
print("Object type is: ", type(studentObject))

print("Student Details")
print(studentObject.rollNumber, studentObject.name, studentObject.marks.english, studentObject.marks.geometry)

Encode Object into JSON formatted Data using jsonpickle
{"py/object": "__main__.Student", "rollNumber": 1, "name": "Emma", "marks": {"py/object": "__main__.Marks", "english": 82, "geometry": 74}}
Decode and Convert JSON into Object using jsonpickle
Object type is:  <class '__main__.Student'>
Student Details
1 Emma 82 74


## Example 6

In [ ]:
import json
from json import JSONEncoder

class Student(object):
    def __init__(self, rollNumber, name, *args, **kwargs):
        self.rollNumber = rollNumber
        self.name = name

class StudentEncoder(JSONEncoder):
        def default(self, o):
            return o.__dict__

student = Student(1, "Emma")

# encode Object
studentJson = json.dumps(student, cls=StudentEncoder, indent=4)

#Decode JSON
resultDict = json.loads(studentJson)

print("Converting JSON into Python Object")
studentObj = Student(**resultDict)

print("Object type is: ", type(studentObj))

print("Student Details")
print(studentObj.rollNumber, studentObj.name)

Converting JSON into Python Object
Object type is:  <class '__main__.Student'>
Student Details
1 Emma
